In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from atm_transport import energy_flux

In [3]:
"""
Constants used in model
"""

grav = 9.80  # Gravitational acceleration
rdgas = 287.04 # Gas constant for dry air
kappa = 2./7.  # rdgas/cp_air
cp_air = rdgas/kappa  # Specific heat capacity of dry air at constant pressure
a = 6376.0e3  # Earth radius
stefan = 5.6734e-8   # Stefan-Boltzmann constant
rvgas = 461.50 # Gas constant for water vapour
cp_vapor = 4.0*rvgas # specific heat capacity of water vapor at constant pressure
dens_h20 = 1000. # density of liquid water
L = 2.500e6  # Latent heat of vaporisation
rhoair = 1.292269  # reference atmospheric density
omega = 7.2921150e-5
pstd_mks = 101325.0
tfreeze = 273.16

In [9]:
ctl_300 = xr.open_dataset('./../data_isca/t42_300_ga7_alb0.3_clim.nc')
ctl_1200 = xr.open_dataset('./../data_isca/t42_1200_ga7_alb0.3_clim.nc')
ctl_4800 = xr.open_dataset('./../data_isca/t42_4800_ga7_alb0.3_clim.nc')
ctl_300 = ctl_300.where(ctl_300<1e19,np.nan)
ctl_1200 = ctl_1200.where(ctl_1200<1e19,np.nan)
ctl_4800 = ctl_4800.where(ctl_4800<1e19,np.nan)

In [18]:
import gradients as gr

def mse_budg(data):
      
    # Evaluate upward radiative fluxes at the surface
    data['rflux_surf'] = data.t_surf ** 4. * stefan - data.soc_surf_flux_sw - data.soc_surf_flux_lw
    # Evaluate downward radiative fluxes at the TOA
    data['rflux_toa'] = data.soc_toa_sw - data.soc_olr
    
    # Evaluate net flux of energy into column
    Fnet = data.flux_lhe + data.flux_t + data.rflux_surf + data.rflux_toa
    
    # Evaluate moist static energy (cpT + Lq + gz) flux into the column by mean flow
    mse = cp_air * data.vcomp_temp + L * data.sphum_v + grav * data.vcomp_height
    
    # Evaluate eddy fluxes
    uT_eddy = data.ucomp_temp - data.ucomp * data.temp
    vT_eddy = data.vcomp_temp - data.vcomp * data.temp
    wT_eddy = data.omega_temp - data.omega * data.temp
    
    uT_dx_eddy = -1.* cp_air * gr.ddx(uT_eddy)
    vT_dy_eddy = -1. * cp_air * gr.ddy(vT_eddy)
    wT_dp_eddy = -1. * cp_air * gr.ddp(wT_eddy)
    
    uq_eddy = data.sphum_u - data.ucomp * data.sphum
    vq_eddy = data.sphum_v - data.vcomp * data.sphum
    wq_eddy = data.sphum_w - data.omega * data.sphum
    
    uq_dx_eddy = -1. * L * gr.ddx(uq_eddy)
    vq_dy_eddy = -1. * L * gr.ddy(vq_eddy)
    wq_dp_eddy = -1. * L * gr.ddp(wq_eddy)
    
    uz_eddy = data.ucomp_height - data.ucomp * data.height
    vz_eddy = data.vcomp_height - data.vcomp * data.height
    wz_eddy = data.omega_height - data.omega * data.height
    
    uz_dx_eddy = -1. * grav * gr.ddx(uz_eddy)
    vz_dy_eddy = -1. * grav * gr.ddy(vz_eddy)
    wz_dp_eddy = -1. * grav * gr.ddp(wz_eddy)
    
    eddies = (uT_dx_eddy + vT_dy_eddy + wT_dp_eddy + 
              uq_dx_eddy + vq_dy_eddy + wq_dp_eddy +
              uz_dx_eddy + vz_dy_eddy + wz_dp_eddy)
    
    u_dmsedx_mean = -1. * (data.ucomp * gr.ddx(mse))
    v_dmsedy_mean = -1. * (data.vcomp * gr.ddy(mse, vector=False))
    w_dmsedp_mean = -1. * (data.omega * gr.ddp(mse))
    
    # Evaluate rate of change of internal plus potential energy (cvT + Lq + gz)
    e = ((cp_air - rdgas) * data.temp + L * data.sphum + grav * data.height)
    dedt = gr.ddt(e)
    
    # Take column integrals of dedt, advective fluxes, and mse
    def column_int(var_in):
        var_int = var_in.sel(pfull=np.arange(50.,1000.,50.)).sum('pfull')*5000./grav
        return var_int
    
    u_dmsedx_mean = column_int(u_dmsedx_mean)
    v_dmsedy_mean = column_int(v_dmsedy_mean)
    w_dmsedp_mean = column_int(w_dmsedp_mean)
    eddies = column_int(eddies)
    dedt = column_int(dedt)
    mse = column_int(mse)
    
    # Approximate residual as eddy flux to close budget
    residual = Fnet - dedt + u_dmsedx_mean + v_dmsedy_mean + w_dmsedp_mean + eddies
    
    return mse

In [19]:
mse = mse_budg(ctl_300)

In [ ]:
import numpy.matlib
def my_integration(f_,sigma_full_):

    # define sigma on half levels
    num_lev_    = sigma_full_.shape[0]
    sigma_half_ = np.zeros(num_lev_+1)

    #  lowest half level is ground
    sigma_half_[num_lev_] = 1
    
    sigma_half_[0]=0

    for j in range(num_lev_-1,-1,-1):
        sigma_half_[j] = 2*sigma_full_[j] - sigma_half_[j+1]
    
    sigma_half_[0]=0 
    d_sigma_  = np.diff(sigma_half_)
    d_temp_   = np.transpose(d_sigma_).reshape(f_.shape[0],1)
    d_sigma2_ = np.matlib.repmat(d_temp_,1,f_.shape[1])
    
    fin=np.sum(f_*d_sigma2_,0)
    
    return fin

In [ ]:
def penergy_flux_vs_lat(data):
    
    temp          = data.temp.mean(('months','lon'))
    sig           = data.pfull/1000.0
    lat           = data.lat
    z_flux        = data.vcomp_height.mean(('months','lon'))
    temp_flux     = data.vcomp_temp.mean(('months','lon'))
    sh_flux       = data.sphum_v.mean(('months','lon'))
    
    # set parameters
    deg             = np.pi/180
    l_cond          = 2500000
    radius          = 6371000
    p0              = 100000
    grav            = 9.8000
    cp              = 1.00464e+03
    
    lati, sigi = np.meshgrid(lat,sig)
    # temp and z fluxes include a cos(lat) factor
    temp_flux = temp_flux/np.cos(lati*deg)
    z_flux = z_flux/np.cos(lati*deg)
    
    # variables to be plotted
    energy_flux =  cp*temp_flux + grav*z_flux + l_cond*sh_flux
    dry_energy_flux =  cp*temp_flux+grav*z_flux
    hum_energy_flux =  l_cond*sh_flux
        
    # now integrate over sigma
    energy_flux_1d = my_integration(energy_flux, sig)
    dry_energy_flux_1d = my_integration(dry_energy_flux, sig)
    hum_energy_flux_1d = my_integration(hum_energy_flux, sig)
    
    # convert to power in PW
    rescale = 2.0*np.pi*radius*p0/grav*np.cos(lat*deg)/1e15
    energy_flux_1d = energy_flux_1d*rescale
    dry_energy_flux_1d = dry_energy_flux_1d*rescale
    hum_energy_flux_1d = hum_energy_flux_1d*rescale
    
    return energy_flux_1d ,dry_energy_flux_1d,hum_energy_flux_1d #,z_energy_flux_1d

In [ ]:
[mse,dry,hum] = penergy_flux_vs_lat(ctl_300)

In [ ]:
hum.plot()

In [ ]:
test_et = xr.open_dataset('./../data_isca/aquaplanet_p2alb_ga7_ml2_ave_for_et.nc')
pdiff=np.zeros(test_et.pfull.values.shape[0]+1)
pdiff[0]=0
pdiff[test_et.pfull.values.shape[0]]=1000
for i in np.arange(1,test_et.pfull.values.shape[0]):
    pdiff[i] = (test_et.pfull.values[i-1]+test_et.pfull.values[i])/2
np.diff(pdiff)

grav = 9.80  # Gravitational acceleration
rdgas = 287.04 # Gas constant for dry air
kappa = 2./7.  # rdgas/cp_air
cp_air = rdgas/kappa  # Specific heat capacity of dry air at constant pressure
a = 6376.0e3  # Earth radius
L = 2.500e6  # Latent heat of vaporisation

data = test_et

# Evaluate moist static energy (cpT + Lq + gz) flux into the column by mean flow
mse_2d = cp_air * data.vcomp_temp + L * data.sphum_v +grav * data.vcomp_height
moist_2d = L * data.sphum_v
dry_2d = cp_air * data.vcomp_temp
height_2d = grav * data.vcomp_height

pfull_diff = data.pfull
pfull_diff = pfull_diff.where(pfull_diff.pfull>800,5000.)
pfull_diff = pfull_diff.where(pfull_diff.pfull<=800,2500.)


mse = (mse_2d*pfull_diff*np.cos(data.lat*np.pi/180)).where(data.pfull>50).sum(('pfull'))*2*np.pi*a/grav/1e15
moist = (moist_2d*pfull_diff*np.cos(data.lat*np.pi/180)).where(data.pfull>50).sum(('pfull'))*2*np.pi*a/grav/1e15
dry = (dry_2d*pfull_diff*np.cos(data.lat*np.pi/180)).where(data.pfull>50).sum(('pfull'))*2*np.pi*a/grav/1e15
height = (height_2d*pfull_diff*np.cos(data.lat*np.pi/180)).where(data.pfull>50).sum(('pfull'))*2*np.pi*a/grav/1e15